In [1]:
from dataloader.LAV_DF import create_lav_df_dataloader, get_train_list, get_lav_df_metadata
from dataloader.LAV_DF.utils import read_json
from localize_fakes import run_inference, extract_fake_periods
import numpy as np
data_root = "DATASET/LAV-DF/LAV-DF/LAV-DF"

c:\Users\nicla\anaconda3\envs\PL_MAI\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\nicla\anaconda3\envs\PL_MAI\Lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
c:\Users\nicla\anaconda3\envs\PL_MAI\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [3]:
train_list = get_train_list(data_root)
train_list[0], len(train_list[0])

([{'file_id': '000001',
   'video_path': 'DATASET/LAV-DF/LAV-DF/LAV-DF\\test/000001.mp4',
   'file': 'test/000001.mp4',
   'n_fakes': 0,
   'fake_periods': [],
   'timestamps': [['not', 0.0, 0.2],
    ['the', 0.2, 0.4],
    ['point', 0.4, 0.8],
    ['the', 0.8, 1.7],
    ['point', 1.7, 1.7],
    ['is', 1.7, 2.1],
    ['that', 2.1, 2.3],
    ['she', 2.3, 2.5],
    ['died', 2.5, 2.7],
    ['satisfied', 2.7, 3.6],
    ['with', 3.6, 3.8],
    ['the', 3.8, 3.8],
    ['life', 3.8, 4.0]],
   'duration': 4.224,
   'transcript': 'not the point the point is that she died satisfied with the life',
   'original': None,
   'modify_video': False,
   'modify_audio': False,
   'split': 'test',
   'video_frames': 103,
   'audio_channels': 1,
   'audio_frames': 65536},
  {'file_id': '000000',
   'video_path': 'DATASET/LAV-DF/LAV-DF/LAV-DF\\test/000000.mp4',
   'file': 'test/000000.mp4',
   'n_fakes': 0,
   'fake_periods': [],
   'timestamps': [['I', 0.0, 0.1],
    ['side', 0.1, 0.4],
    ['with', 0.4, 0

In [2]:
fake_preds = read_json("fake_preds.json")
real_preds = read_json("real_preds.json")

In [3]:
import numpy as np
from sklearn.metrics import roc_auc_score

def compute_auc(real_distances: np.ndarray, 
                fake_distances: np.ndarray,
                aggregate: str = 'mean') -> float:
    """
    Computes the AUC for a deepfake detection model based on distance scores.

    Parameters
    ----------
    real_distances : np.ndarray
        An array of shape (N_real, video_length) containing the frame-level
        distances for real videos.
    fake_distances : np.ndarray
        An array of shape (N_fake, video_length) containing the frame-level
        distances for fake videos.
    aggregate : str
        How to aggregate frame-level distances into a single score per video.
        Options:
          - 'mean': Use the average distance across all frames
          - 'max' : Use the maximum distance across all frames
          - 'min' : Use the minimum distance across all frames
          - 'median': Use the median distance across all frames

    Returns
    -------
    float
        The AUC for the given real and fake distance distributions.
    """

    # Choose an aggregation function
    if aggregate == 'mean':
        agg_func = np.mean
    elif aggregate == 'max':
        agg_func = np.max
    elif aggregate == 'min':
        agg_func = np.min
    elif aggregate == 'median':
        agg_func = np.median
    else:
        raise ValueError(f"Unknown aggregate function: {aggregate}")

    # Aggregate distances to a single score per video
    real_scores = agg_func(real_distances, axis=1)
    fake_scores = agg_func(fake_distances, axis=1)

    # Create ground truth labels: 0 for real, 1 for fake
    labels = np.concatenate([np.zeros(len(real_scores)), np.ones(len(fake_scores))])

    # Concatenate all scores
    scores = np.concatenate([real_scores, fake_scores])

    # Compute AUC
    auc = roc_auc_score(labels, scores)
    return auc


fake_distances = np.array([pred["distance"] for pred in fake_preds])
real_distances = np.array([pred["distance"] for pred in real_preds])

auc_value = compute_auc(real_distances, fake_distances, aggregate='mean')
auc_value_max = compute_auc(real_distances, fake_distances, aggregate='max')
auc_value_min = compute_auc(real_distances, fake_distances, aggregate='min')
auc_value_median = compute_auc(real_distances, fake_distances, aggregate='median')

print(f"Computed AUC (mean aggregation): {auc_value:.4f}")
print(f"Computed AUC (max aggregation): {auc_value_max:.4f}")
print(f"Computed AUC (min aggregation): {auc_value_min:.4f}")
print(f"Computed AUC (median aggregation): {auc_value_median:.4f}")



Computed AUC (mean aggregation): 0.5621
Computed AUC (max aggregation): 0.4714
Computed AUC (min aggregation): 0.5735
Computed AUC (median aggregation): 0.5077
